In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
img_array = cv2.imread("Dataset/Full_Dataset/Closed_Eye/s0001_00056_0_0_0_0_0_01.png", cv2.IMREAD_GRAYSCALE)

In [ ]:
plt.imshow(img_array, cmap="gray")

In [ ]:
img_array.shape

In [ ]:
datadir = "Dataset/Test_Dataset"
classes = ["Closed_Eye", "Opened_Eye"]
for category in classes:
    path = os.path.join(datadir, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        backtorgb= cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
        plt.imshow(img_array, cmap="gray")
        plt.show()
        break
    break

In [ ]:
img_size = 224
new_array = cv2.resize(backtorgb, (img_size, img_size))
plt.imshow(new_array, cmap='gray')
plt.show()

In [ ]:
training_data = []
def create_training_data():
    for category in classes:
        path = os.path.join(datadir, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
                new_array = cv2.resize(backtorgb, (img_size, img_size))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass

create_training_data()

In [ ]:
print(len(training_data))

In [ ]:
import random 

random.shuffle(training_data)

In [ ]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, img_size, img_size, 3)


In [ ]:
X.shape

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# X = scaler.fit_transform(X.reshape(-1, 224*224*3)).reshape(X.shape)

X = X / 255.0

In [ ]:
y = np.array(y)
y.shape

In [ ]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
model = tf.keras.applications.mobilenet.MobileNet()


In [ ]:
model.summary()

# Transfer Learning

In [ ]:
base_input = model.layers[1].input
base_input

In [ ]:
base_output = model.layers[-4].output
base_output

In [ ]:
flat_layers = layers.Flatten()(base_output)
final_output = layers.Dense(1)(flat_layers)
final_output = layers.Activation('sigmoid')(final_output)

In [ ]:
new_model = keras.Model(inputs = base_input, outputs = final_output)

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(loss="binary_crossentropy", optimizer= "adam", metrics=["accuracy"])

In [ ]:
new_model.fit(X, y, epochs = 3, validation_split = 0.2) ##training

In [ ]:
model.save('my_model.keras')

In [ ]:
img_array = cv2.imread("Dataset/Train_Dataset/Opened_Eye/s0001_01845_0_0_1_0_0_01.png", cv2.IMREAD_GRAYSCALE)
backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
new_array = cv2.resize(backtorgb, (img_size, img_size))
X_input = np.array(new_array).reshape(1, img_size, img_size, 3)


In [ ]:
X_input.shape

In [ ]:
X_input = X_input/255.0 #normalizing data


In [ ]:
prediction = new_model.predict(X_input)

In [ ]:
prediction

In [2]:
# Load the saved model
import tensorflow as tf
new_model = tf.keras.models.load_model("my_model.keras")

In [3]:
# import winsound   # For windows alerting
frequency = 2500  # Set frequency to 2500
duration = 1500  # Set duration to 1500 ms == 1.5 sec
import numpy as np
import cv2
path = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
cap = cv2.VideoCapture(0)

#check if webcam is opened correctly
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    print("resetting camera")
else:
    print("Camera initialised")
cap.set(cv2.CAP_PROP_FPS, 5)
counter = 0

while True:
    ret,frame = cap.read()
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
    for x,y,w,h in eyes:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0, 255, 0), 2)
        eyess = eye_cascade.detectMultiScale(roi_gray)
        if len(eyess) == 0:
            print("Eyes are not detected")
        else:
            for (ex, ey, ew, eh) in eyess:
                eyes_roi = roi_color[ey: ey+eh, ex: ex+ew]
                
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if(faceCascade.empty()==False):
        print("detected")
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    # Draw a rectangle around eyes
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    font = cv2.FONT_HERSHEY_SIMPLEX

    final_image = cv2.resize(eyes_roi, (224,224))
    final_image = np.expand_dims(final_image, axis=0)
    final_image = final_image/255.0
    Predictions = new_model.predict(final_image)
    if np.any(Predictions >= 0.3):
        status = "Open Eyes"
        cv2.putText(frame,
                status,
                (150,150),
                font, 3,
                (0, 255, 0),
                2,
                cv2.LINE_4)
        x1,y1,w1,h1 = 0,0,175,75
        cv2.rectangle(frame, (x1, y1), (x1 + w1, y1 + h1), (0,0,0), -1)
        #Add text
        cv2.putText(frame, 'Active', (x1 + int(w1/10),y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    elif np.any(Predictions<0.3):
        counter = counter + 1
        status = "Closed Eyes"
        cv2.putText(frame,
                status,
                (150,150),
                font, 3,
                (0, 0, 255),
                2,
                cv2.LINE_4)
        x1,y1,w1,h1 = 0,0,175,75
        cv2.rectangle(frame, (x1,y1), (x1 + w1, y1 + h1), (0,0,255), 2)
        if counter > 10:
            x1,y1,w1,h1 = 0,0,175,75
            #Draw black background rectangle
            cv2.rectangle(frame, (x1, y1), (x1 + w1, y1 + h1), (0,0,0), -1)
            #Add text
            cv2.putText(frame, "Sleep Alert !!!", (x1 + int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            # winsound.Beep(frequency, duration)    #For windows alerting
            counter = 0
    cv2.imshow("Drowsiness Detection", frame)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Camera initialised
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
Eyes are not detected
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Eyes are not detected
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Eyes are not detected
Eyes are not detected
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Eyes are not detected
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Eyes are not detected
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Eyes are not detected
Eyes are not detected
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Eyes are not detected
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Eyes are not detected
Eyes are not detected
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Eyes are not detected
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Eyes are not detected
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Eyes are not detected
Eyes are not detected
detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16

KeyboardInterrupt: 